# Nim with a Quantum Computer

[Nim](https://en.wikipedia.org/wiki/Nim) is a simple game for which we can implement simple AIs. For that reason it was used as the basis of some of the first examples of computer games: such as 1940's [Nimatron](https://en.wikipedia.org/wiki/Nimatron) and 1951's [Nimrod](https://en.wikipedia.org/wiki/Nimrod_(computer)). There was even [Dr Nim](https://en.wikipedia.org/wiki/Dr._Nim): a simple mechanical version made in the 1960s.

Now it's time to do it with quantum computing. In this notebook we'll implement a Nim AI using just a single qubit. The game can be played here, or on Twitter.

First we import the things we'll need: `numpy`, various things from `qiskit` and some `twitter_tools` made specifically for this project (which can be found in the file [twitter_tools.py](twitter_tools.py)).

In [ ]:
import numpy as np
from twitter_tools import *

from qiskit import QuantumCircuit, execute, IBMQ, Aer
from qiskit.providers.ibmq import least_busy

Next is to set up how the game will be played. With `twitter=False`, all inputs and outputs will be restricted to this notebook. With `simulator=True`, a local simulator will be used. But with
```
twitter = True
simulator = False
```
the inputs and outputs really will be done via Twitter, and real quantum hardware will be used to run the opponent AI.

In [ ]:
twitter = False
simulator = True

if simulator:
    backend = Aer.get_backend('qasm_simulator')
else:
    IBMQ.load_account()
    provider = IBMQ.providers()[0]
    backend = least_busy(provider.backends()[1::])
    
print('Playing against '+backend.name())

Now for the actual game!

There are initially 12 marbles. Players take it in turns to remove 1, 2 or 3 marbles. Whoever takes the last one wins.

There is a strategy by which the second player can always win. They simply need to ensure that each pair of turns results in four marbles being taken. With this tactic, after 6 moves (3 by each player) the game will be down to just four marbles.

At this point, it will be the turn of Player 1. However many marbles they take, it will be possible for Player 2 to take the rest. This mean taking the last marble, and so winning the game!

To implement an AI for Player 2, we mostly just need to be able to determine when the number of marbles is a multiple of 4. For a quantum computer, we can do this using one qubit (as long as we can repeat the process for many shots. If you are not familiar with single qubit gates, [this guide](https://nbviewer.jupyter.org/github/quantumjim/blog/blob/master/Quantum_Procedural_Generation/2_SingleQubit.ipynb) should give you all you need for what follows.

For our quantum AI we'll use just one gate: the `rx` gate for an angle of $\pi/4$. To see how, let's look at the effects of apply different numbers of this gate before measurement (the point at which a `0` or `1` is extracted).

* No `rx` gates: The qubit is in the initial $\left|0\right\rangle$ state, and so the measurement outputs `0`.
* One `rx` gates: The output will be randomly `0` or `1` with equal probability.
* Two `rx` gates: The is rotated to the $\left|1\right\rangle$ state, and the measurement outputs `1`.
* Three `rx` gates: The output will be random.
* Four `rx` gates: The qubit will be back to the initial `0` state.

To see this for yourself, you can create these circuits with the code below.

In [ ]:
num_rx = 5

qc = QuantumCircuit(1,1)
for _ in range(num_rx):
    qc.rx(2*np.pi/4,0)
qc.measure(0,0)

qc.draw(output='mpl')

And then run them (on a simulator for simplicity). This is done 1024 times, and the number of samples for each possible output (`0` or `1`) are given.

In [ ]:
execute(qc,Aer.get_backend('qasm_simulator')).result().get_counts()

We will use the variable `nimmed` to represent the number of marbles taken at the beginning of a turn. There are therefore three possible numbers of marbles that could be taken at the end of the turn: `nimmed+1`, `nimmed+2` or `nimmed+3`. The quantum AI then runs three circuits: one with `nimmed+1` `rx` gates, one with `nimmed+2` and one with `nimmed+3`.  It looks at the results of each to see which returns the most `0` outputs. Given the results we saw above, this will be the one such that the total number of marbles taken is a multiple of four (assuming that one of the moves can acheive this).

That's basically it! Just run the following cell to play.

In [ ]:
marbles = 12
nimmed = 0
winner = None

num = 1

thread_id, num = twitter_print('It\'s '+str(time.time())+', which is the perfect time for a game of Nim!\n\n'+
                          'Twitter users will be playing against IBM quantum computers.\n\n'+
                          'The opponent in this game is '+backend.name()+'.\n\n'+
                          'Keep refreshing twitter.com/QubitChannel to follow the action.',
                          num, twitter=twitter)

ai_turn = 'The game begins with twelve marbles.\n\n'+'O'*(marbles-nimmed)+'\n\nPlayers take turns to remove either 1, 2 or 3. Whoever takes the last one wins!'

while winner==None:
        
    # player turn
    turn = True
    while turn:
        nim, author, thread_id, num = twitter_input(ai_turn,thread_id,num,twitter=twitter)
        try:
            nim = int(nim)
            if nim in [1,2,3]:
                nimmed += nim
                turn = False
        except:
            print('That isn\'t a valid move. Try again.')
          
    # check to see if the player wins
    if nimmed==marbles:
         winner=='Twitter users'
            
    # otherwise, its the ai's turn
    else:
        
        # create a circuit corresponding to each move
        qc = {}
        for nim in [1,2,3]:
            qc[nim] = QuantumCircuit(1,1)
            for _ in range(nim+nimmed):
                qc[nim].rx(2*np.pi/4,0)
            qc[nim].measure(0,0)

        # run them and see which gives the best results
        result = execute(list(qc.values()),backend).result()
        zeroes = {}
        for nim in qc:
            counts = result.get_counts(qc[nim])
            if '0' in counts:
                zeroes[nim] = counts['0']
        nim = max(zeroes, key=zeroes.get)
        nimmed += nim
        ai_turn = 'The '+backend.name()+' device has responded by taking '+str(nim)+' marble'+'s'*(nim!=1)+'.\n\n'+'O'*(marbles-nimmed)

        if nimmed==marbles:
            winner = backend.name()
    
thread_id = twitter_print('The last marble was taken by '+winner+'!\nVictory for '+winner+'!',
              num, reply_to=thread_id, twitter=twitter)